In [1]:
import hepfile as hf
import fastq as fq

In [2]:
filepath = '1_control_18S_2019_minq7.fastq'
#filepath = 'fastq-test-input.fastq'

In [3]:
fastq = fq.read(filepath)
fos = list(fastq)

In [4]:
import cProfile
with cProfile.Profile() as profile:
    hepfile = hf.initialize()
    for fo in fos:
        meta = fo.getHead().split()

        # get the sample id, read, and ch as group name b/c I think that is unique 
        group = f"{meta[2].split('=')[-1]}_{meta[3].split('=')[-1]}_{meta[4].split('=')[-1]}"

        # create the group and add metadata for that group
        hf.create_group(hepfile, group, counter=f'n_{group}')
        hf.add_group_meta(hepfile, group, meta)

        # create a dataset underneath that group
        hf.create_dataset(hepfile, 'sequence', group=group, dtype=str)
        hf.create_dataset(hepfile, 'quality', group=group, dtype=str)

    bucket = hf.create_single_bucket(hepfile)
    for fo in fos:

        meta = fo.getHead().split()
        group = f"{meta[2].split('=')[-1]}_{meta[3].split('=')[-1]}_{meta[4].split('=')[-1]}"

        seq = list(fo.getSeq())
        qual = list(fo.getQual())
        bucket[f'{group}/sequence'] = seq
        bucket[f'{group}/quality'] = qual

    return_value = hf.pack(hepfile,bucket,STRICT_CHECKING=True,verbose=False)  
    
    profile.print_stats()

         697593 function calls in 3.681 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    36714    0.004    0.000    0.004    0.000 _fastq.py:45(getHead)
    18357    0.003    0.000    0.004    0.000 _fastq.py:51(getSeq)
    18357    0.003    0.000    0.004    0.000 _fastq.py:57(getQual)
    18357    0.001    0.000    0.001    0.000 _fastq.py:69(body)
    18357    0.001    0.000    0.001    0.000 _fastq.py:85(qstr)
        1    0.000    0.000    0.000    0.000 cProfile.py:40(print_stats)
        1    0.000    0.000    0.000    0.000 cProfile.py:50(create_stats)
        1    0.000    0.000    0.000    0.000 pstats.py:107(__init__)
        1    0.000    0.000    0.000    0.000 pstats.py:117(init)
        1    0.000    0.000    0.000    0.000 pstats.py:136(load_stats)
        1    0.000    0.000    0.000    0.000 write.py:13(initialize)
    36714    0.068    0.000    0.112    0.000 write.py:181(create_dataset)
    18357    0

In [5]:
outfile = 'fastq-test.h5'
hf.write_to_file(outfile, hepfile)

Metadata added


<Closed HDF5 file>

In [6]:
import cProfile
with cProfile.Profile() as profile:
    awk, bucket = hf.load(outfile)#, return_awkward=True)
    profile.print_stats()

Building the indices...

Built the indices!
Data is read in and input file is closed.
         13621709 function calls (13493203 primitive calls) in 53.805 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   128503    0.083    0.000    1.077    0.000 <__array_function__ internals>:2(prod)
    36716    0.023    0.000    0.323    0.000 <__array_function__ internals>:2(product)
        3    0.000    0.000    0.077    0.026 <__array_function__ internals>:2(unique)
    18357    0.034    0.000    0.198    0.000 <__array_function__ internals>:2(where)
   422243    0.172    0.000    0.239    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
   514030    0.178    0.000    0.252    0.000 <frozen importlib._bootstrap>:398(parent)
    18357    0.007    0.000    0.015    0.000 <string>:1(<lambda>)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1689(